- 做資料整理

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

x_train = pd.read_csv('x_train.csv')
print(x_train.shape)
y_train = pd.read_csv('y_train.csv', header=None)
print(y_train.shape)
x_test = pd.read_csv('x_test.csv')
print(x_test.shape)
y_test = pd.read_csv('y_test.csv', header=None)
print(y_test.shape)


(361, 16384)
(361, 1)
(42, 16384)
(42, 1)


In [2]:
x_test = x_test.to_numpy()
x_train = x_train.to_numpy()
# 把它變成一維
y_test = y_test.to_numpy().astype(int).ravel() 
y_train = y_train.to_numpy().astype(int).ravel() 

print("\n--------------new------------\n")
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)



--------------new------------

x_train: (361, 16384)
y_train: (361,)
x_test: (42, 16384)
y_test: (42,)


- 假如單純用knn

In [3]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


In [4]:
# 1. 使用剛剛選出來的最佳 k (best_k) 重新建立模型 pipeline
final_pipe = Pipeline([
    ("knn", KNeighborsClassifier(n_neighbors=1)) 
])

final_pipe.fit(x_train, y_train)


test_score = final_pipe.score(x_test, y_test)

print("-" * 30)
print(f"測試集 (Test Set) 準確率: {test_score:.3f}")

------------------------------
測試集 (Test Set) 準確率: 0.786


- 這邊做lda後做knn

In [5]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# ------------------------------------------------------------------------
# 先做空間轉換
scaler = StandardScaler()


x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


pca = PCA(n_components=0.9) 


x_train_pca = pca.fit_transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)
lda = LinearDiscriminantAnalysis()

x_train_lda = lda.fit_transform(x_train_pca, y_train)# 這邊已經算出線性組合
x_test_lda = lda.transform(x_test_pca)#會套用這邊的

print("原始維度:", x_train.shape[1])
print("LDA 後維度:", x_train_lda.shape[1])

best_k = 1
# ------------------------------------------------------------------------
# 再做knn 
final_pipe = Pipeline([
    ("knn", KNeighborsClassifier(
        n_neighbors=best_k, # 這邊k = 1 用cross validation得出的
    ))
])


final_pipe.fit(x_train_lda, y_train)


y_pred = final_pipe.predict(x_test_lda)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"測試集辨識正確率: {test_accuracy}")


原始維度: 16384
LDA 後維度: 20
測試集辨識正確率: 0.6904761904761905


In [6]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# 假設 x_train, x_test, y_train, y_test 已經定義好

# ------------------------------------------------------------------------
# 將所有步驟整合到一個 Pipeline
# 這樣做的好處是：之後如果要做 GridSearchCV，可以直接對整個流程做，不用擔心資料洩漏
full_pipeline = Pipeline([
    ("pca", PCA(n_components=0.95)),             # 2. PCA 降維
    ("lda", LinearDiscriminantAnalysis()),      # 3. LDA 監督式降維
    ("knn", KNeighborsClassifier(n_neighbors=1))# 4. KNN 分類
])

# ------------------------------------------------------------------------
# 訓練與預測

full_pipeline.fit(x_train, y_train) 

y_pred = full_pipeline.predict(x_test)
test_accuracy = accuracy_score(y_test, y_pred)

# ------------------------------------------------------------------------
# 查看中間維度 (如果需要 Debug)
# 可以透過 named_steps 取得中間的模型資訊
pca_step = full_pipeline.named_steps['pca']
lda_step = full_pipeline.named_steps['lda']

print(f"原始維度: {x_train.shape[1]}")
print(f"PCA 保留成分數: {pca_step.n_components_}") 
# 注意：LDA 降維後的維度上限是 min(n_features, n_classes - 1)
# 這裡無法直接顯示 LDA output shape，因為 Pipeline 內部處理掉了，但邏輯不變。

print(f"測試集辨識正確率: {test_accuracy}")

原始維度: 16384
PCA 保留成分數: 44
測試集辨識正確率: 1.0
